In [1]:
import numpy as np
import mpmath as mp
import math as mt
import matplotlib as mpl
import pandas as pd
import scipy as sc

#Constantes físicas y parámetros del experimento

h=4.135667696e-15
hb=6.582119569e-16
kb=8.617333262e-5
m=3.8175458e-26
v=0.001**3
n0=1e6
t0=1e-6

In [2]:
def newtonraphson_1var(x1,x2,dx,tol,m,n,t):
    l=mt.sqrt((2*mt.pi*hb**2)/(m*kb*t))
    def f(alpha_0):
        return ((v)/(l**3))*mp.polylog(3/2,mp.exp(alpha_0))
    def df(alpha_0):
        return ((v)/(l**3))*mp.polylog(1/2,mp.exp(alpha_0))
    while x1<x2:
        f_n=f(x1)
        df_n=f(x1+dx)
        b=f_n*df_n
        if b<0:
            x_n=x1
            c=1
            while True:
                x_nm1=x_n
                x_np1=x_n-f(x_n)/df(x_n)
                x_n=x_np1
                c+=1
                if abs(x_n-x_nm1<tol) or step==m:
                    break
            return x_n*kb*t0
        x1=x1+dx

In [22]:
l=mt.sqrt((2*mt.pi*hb**2)/(m*kb*t0))
def f(alpha_0):
    return ((v)/(l**3))*mp.polylog(3/2,mp.exp(alpha_0))-n0
def df(alpha_0):
    return ((v)/(l**3))*mp.polylog(1/2,mp.exp(alpha_0))
for i in np.arange(-1e-6, 0, 1e-8):
    print(i,kb*t0*((v)/(l**3))*mp.polylog(3/2,mp.exp(i)),((v)/(l**3))*mp.polylog(1/2,mp.exp(i)))

-1e-06 2.9866724678533e-28 2.35280307705668e-15
-9.9e-07 2.98669279372579e-28 2.36466583823433e-15
-9.800000000000001e-07 2.98671322260074e-28 2.37670971222287e-15
-9.700000000000002e-07 2.98673375605857e-28 2.38893935481299e-15
-9.600000000000003e-07 2.98675439572124e-28 2.4013595915135e-15
-9.500000000000004e-07 2.98677514325212e-28 2.41397542459474e-15
-9.400000000000005e-07 2.98679600035942e-28 2.42679204265783e-15
-9.300000000000006e-07 2.98681696879594e-28 2.43981482847915e-15
-9.200000000000007e-07 2.98683805036247e-28 2.45304936950058e-15
-9.100000000000008e-07 2.98685924690884e-28 2.46650146748557e-15
-9.000000000000009e-07 2.98688056033515e-28 2.48017714874676e-15
-8.90000000000001e-07 2.98690199259512e-28 2.49408267638673e-15
-8.800000000000011e-07 2.98692354569707e-28 2.50822456159656e-15
-8.700000000000012e-07 2.98694522170723e-28 2.52260957724626e-15
-8.600000000000013e-07 2.98696702275065e-28 2.53724477057133e-15
-8.500000000000014e-07 2.98698895101436e-28 2.552137478139

In [6]:
evap_cool={'N':[],'T':[],'E':[],'Mu':[],'Q':[]}
#mu_0=newtonraphson_1var(-12.0902,-12.0901,1e-30,1e-35,100000,n0,t0)
mu_0=-12.0902
print(mu_0)
E_0=(3/2)*n0*t0*kb*(mp.polylog(5/2,mp.exp(mu_0))/mp.polylog(3/2,mp.exp(mu_0)))
evap_cool['N'].append(n0)
evap_cool['T'].append(t0)
evap_cool['E'].append(E_0)
evap_cool['Mu'].append(mu_0)
#Cortes en temperatura:
qc=mt.sqrt(2*m*E_0)
dqc=0.01e-6
for i in range(0,1000):
    evap_cool['Q'].append(qc-i*dqc)
#print(len(evap_cool['Q']),len(evap_cool['N']),len(evap_cool['T']),len(evap_cool['E']),len(evap_cool['Mu']))
#print(evap_cool['Q'])

-12.0902


In [7]:
def newton_raphson_2var(x1,x2,y1,y2,dx,dy,tol,n_i,e_i):
    def n(t_i,a_i):#función del número de átomos
        l=mt.sqrt((2*mt.pi*hb**2)/(m*kb*t_i))
        return ((v)/(l**3))*mp.polylog(3/2,mp.exp(a_i))-n_i
    def e(t_i,a_i):#función de la energía
        return (3/2)*n_i*t_i*kb*(mp.polylog(5/2,mp.exp(a_i))/mp.polylog(3/2,mp.exp(a_i)))
    def dn_t(t_i,a_i):#derivada del número de átomos respecto a t_i
        a=(2*mt.pi*m*kb)/h**2
        return (3/2)*(a**(3/2))*mt.sqrt(t_i)*v*mp.polylog(3/2,mp.exp(a_i))
    def dn_a(t_i,a_i):#derivada del número de átomos respecto a a_i
        l=mt.sqrt((2*mt.pi*hb**2)/(m*kb*t_i))
        return (v/l**3)*mp.polylog(1/2,mp.exp(a_i))
    def de_t(t_i,a_i):#derivada de la energía respecto a t_i
        return (3/2)*n_i*kb*(mp.polylog(5/2,mp.exp(a_i))/mp.polylog(3/2,mp.exp(a_i)))
    def de_a(t_i,a_i):#dervida de la energía respecto a a_i
        a=(mp.polylog(5/2,mp.exp(a_i))/(mp.polylog(3/2,mp.exp(a_i))**2))*mp.polylog(-1/2,mp.exp(a_i))
        return (3/2)*n_i*kb*t_i*(1-a)
    while x1>x2 and y1<y2:
        f_n=n(x1,y1)
        df_n=n(x1+dx,y1+dy)
        bracketf=f_n*df_n
        g_n=e(x1,y1)
        dg_n=e(x1+dx,y1+dy)
        bracketg=g_n*dg_n
        if bracketf<0.0:
            x_n=x1
            y_n=y1
            c=1
            while True:
                x_nm1=x_n
                y_nm=y_n
                x_np1=x_n+((dn_a(x_n,y_n)*e(x_n,y_n)-n(x_n,y_n)*de_a(x_n,y_n))/(dn_t(x_n,y_n)*de_a(x_n,y_n)-dn_a(x_n,y_n)*de_t(x_n,y_n)))
                y_np1=y_n+((n(x_n,y_n)*de_t(x_n,y_n)-dn_t(x_n,y_n)*e(x_n,y_n))/(dn_t(x_n,y_n)*de_a(x_n,y_n)-dn_a(x_n,y_n)*de_t(x_n,y_n)))
                x_n=x_np1
                y_n=y_np1
                c+=1
                if abs(x_n-x_nm1)<tol and abs(y_n-y_nm1)<tol:
                    break
            resultado=[x_n,y_n*x_n*kb]
            return resultado
        x1=x1-dx
        y1=y1+dy

In [8]:
#Proceso Recursivo:
def n_regresivo(n_i,t_i,mu_i,qc):
    eta_c=qc**2/(2*m*kb*t_i)
    alpha_i=mu_i/(kb*t_i)
    g32t=mp.polylog(3/2,mp.exp(alpha_i)*mp.erf(eta_c))
    g1b=mp.polylog(1,mp.exp(alpha_i-eta_c))
    g32=mp.polylog(3/2,mp.exp(alpha_i))
    return ((g32t/g32)-(2/mt.sqrt(mt.pi))*mt.sqrt(eta_c)*(g1b/g32))*n_i
def e_regresivo(n_i,t_i,mu_i,e_i,qc):
    eta_c=qc**2/(2*m*kb*t_i)
    alpha_i=mu_i/(kb*t_i)
    g52t=mp.polylog(5/2,mp.exp(alpha_i)*mp.erf(eta_c))
    g52=mp.polylog(5/2,mp.exp(alpha_i))
    g2b=mp.polylog(2,mp.exp(alpha_i-eta_c))
    g1b=mp.polylog(1,mp.exp(alpha_i-eta_c))
    return ((g52t/g52)-(2/mt.sqrt(mt.pi))*mt.sqrt(eta_c)*(g2b/g52)-(4/(3*mt.sqrt(mt.pi)))*(eta_c**(3/2))*(g1b/g52))*n_i

In [9]:
eta_c=evap_cool['Q'][0]**2/(2*m*kb*evap_cool['T'][0])
alpha_i=evap_cool['Mu'][0]/(kb*evap_cool['T'][0])
print(eta_c,alpha_i)
print(mp.exp(alpha_i),mp.erf(eta_c))
print(mp.exp(alpha_i)*mp.erf(eta_c),mp.polylog(3/2,mp.exp(alpha_i)*mp.erf(eta_c)))
print(n_regresivo(evap_cool['N'][0],evap_cool['T'][0],evap_cool['Mu'][0],evap_cool['Q'][0]),e_regresivo(evap_cool['N'][0],evap_cool['T'][0],evap_cool['Mu'][0],evap_cool['E'][0],evap_cool['Q'][0]))

1499998.511290251 -140300944995.52847
4.26903339967307e-60931926218 1.0
4.26903339967307e-60931926218 4.26903339967307e-60931926218
1000000.0 1000000.0


In [10]:
for i in range(1,1001):
    evap_cool['N'].append(n_regresivo(evap_cool['N'][i-1],evap_cool['T'][0],evap_cool['Mu'][0],evap_cool['Q'][i-1]))
    evap_cool['E'].append(e_regresivo(evap_cool['N'][i-1],evap_cool['T'][0],evap_cool['Mu'][0],evap_cool['E'][0],evap_cool['Q'][i-1]))
    ti=newton_raphson_2var(evap_cool['T'][0],1e-9,-11,-1,1e-8,1e-8,1e-10,evap_cool['N'][i],evap_cool['N'][i])[0]
    mui=mewton_raphson_2var(evap_cool['T'][0],1e-9,-11,-1,1e-8,1e-8,1e-10,evap_cool['N'][i],evap_cool['N'][i])[1]
print(ti,mui)

TypeError: 'NoneType' object is not subscriptable